<a href="https://colab.research.google.com/github/dainis-boumber/av/blob/master/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ULMFiT + Siamese Network for Sentence Vectors
## Part Three: Classifying

The second notebook created a new language model from the SNLI dataset.
This notebook will adapt that model to predicting the SNLI category for sentence pairs.
The model will be used as a sentence encoder for a Siamese Network that builds sentence vectors that are feed into a classifier network.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from ipyexperiments import *
import fastai
from fastai.text import *
import html
import json
import html
import re
import pickle
from collections import Counter
import random
import pandas as pd
import numpy as np
from pathlib import Path
import sklearn
from sklearn import model_selection
from functools import partial
from collections import Counter, defaultdict
import random

import numpy as np
import torch
import torch.nn as nn
import torch.utils 
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import dataset, dataloader
import torch.optim as optim
import torch.nn.functional as F

import time
import math
import sys
import data
import joblib

token_files = './data/PAN14/tokens/'
model_files = './data/PAN14/models/'
TRAINDATAPATH = "./data/PAN14/pan14_train_english-essays/"
TESTDATAPATH = "./data/PAN14/pan14_test02_english-essays/"
FNAMES = ['known01','known02','known03','known04','known05', 'unknown']
KCOLS=['known01','known02','known03','known04','known05']
LABELCOL="answer"
UNKOWN="unknown"

In [ ]:
doc_pairs_train=joblib.load(f'{model_files}traindf-2.pkl')
doc_pairs_val=joblib.load(f'{model_files}valdf-2.pkl')
doc_pairs_test=joblib.load(f'{model_files}testdf-2.pkl')

In [ ]:
data_lm = TextLMDataBunch.load(model_files)
data_clf1 = TextClasDataBunch.from_df(model_files, doc_pairs_train, doc_pairs_val, doc_pairs_test,
                                vocab=data_lm.train_ds.vocab, bs=64,
                                  text_cols=['known', 'unknown'], label_cols=['label'], mark_fields=True)

data_clf1.save()          
data_clf1 = TextClasDataBunch.load(model_files, bs=64)
learn1 = text_classifier_learner(data_clf1, drop_mult=0.5)
learn1.load_encoder('healthy_enc')
learn1.lr_find()
learn1.recorder.plot()


exp1=IPyExperimentsPytorch()
learn1.fit_one_cycle(1, slice(1e-03, 1e-02), wd=1e-05)
learn1.fit_one_cycle(1, slice(1e-03/10, 1e-02), wd=1e-04)
learn1.fit_one_cycle(2, slice(1e-04/100, 1e-02), wd=1e-03)

learn1.save('init_av_clf1')

In [ ]:
data_lm = TextLMDataBunch.load(model_files)
data_clf2 = TextClasDataBunch.from_df(model_files, doc_pairs_train, doc_pairs_val, doc_pairs_test,
                                vocab=data_lm.train_ds.vocab, bs=64,
                                  text_cols=['known', 'unknown'], 
                                      label_cols=['label'], 
                                      mark_fields=True
                                     qrnn=True)

data_clf2.save()          
data_clf2 = TextClasDataBunch.load(model_files, bs=64)
learn2 = text_classifier_learner(data_clf2, drop_mult=0.5)
learn2.load_encoder('healthy_enc')
learn2.lr_find()
learn2.recorder.plot()


exp2=IPyExperimentsPytorch()
learn2.fit_one_cycle(1, slice(1e-03, 1e-02), wd=1e-05)
learn2.fit_one_cycle(1, slice(1e-03/10, 1e-02), wd=1e-04)
learn2.fit_one_cycle(2, slice(1e-04/100, 1e-02), wd=1e-03)
learn2.save('data_clf2')

In [ ]:
preds1, y1 = np.array(learn1.get_preds())
preds2, y2 = np.array(learn2.get_preds())
preds1b, y1b = np.load('data_clf1b')
preds2b, y2b = np.load('data_clf2b')

# all y are == so
y = y1
preds = np.hstack((preds1, preds2, preds1b, preds2b))

In [ ]:
from sklearn.linear_model import LogisticRegression

def m(y):
    def proba(y_i, y):
        p = x[y==y_i].sum(0)
        return (p+1) / ((y==y_i).sum()+1)
    y = y.values
    r = np.log(proba(1,y) / proba(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

probas = np.zeros((len(test),1))
lbls = np.zeros((len(test),1))
m,r = get_mdl(train)
probas[:,0] = m.predict_proba(test_x.multiply(r))[:,0]
mu = np.average(probas) # we can do average here or we can go with 0.5 or wecan just report 
# probabilities like recommended by the PAN, whichever works best for specific validation 
# set
for i, prb in enumerate(probas):
    if prb > mu:
        lbls[i] = 1
    else:
        lbls[i] = 0

